In [1]:
# Import necessary libraries
import torch
from squeezenet import FireModule

# Define model parameters
in_channels = 64
squeeze_channels = 16
expansionX1_channels = 64
expansionX3_channels = 64
batch_size = 32
height, width = 128, 128

# Create a random input tensor
x = torch.randn(batch_size, in_channels, height, width)

# Initialize the FireModule
fire_module = FireModule(in_channels, squeeze_channels, expansionX1_channels, expansionX3_channels)

# Forward pass
output = fire_module(x)

# Check the output shape
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([32, 64, 128, 128])
Output shape: torch.Size([32, 128, 128, 128])


In [1]:
from torchsummary import summary
import torch
from squeezenet import SqueezeNet

# Create a SqueezeNet model
model = SqueezeNet(in_channels=3, num_classes=1000)

# Generate model summary
summary(model, input_size=(3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 111, 111]          14,208
              ReLU-2         [-1, 96, 111, 111]               0
         MaxPool2d-3           [-1, 96, 55, 55]               0
            Conv2d-4           [-1, 16, 55, 55]           1,552
              ReLU-5           [-1, 16, 55, 55]               0
            Conv2d-6           [-1, 64, 55, 55]           1,088
              ReLU-7           [-1, 64, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]           9,280
              ReLU-9           [-1, 64, 55, 55]               0
       FireModule-10          [-1, 128, 55, 55]               0
           Conv2d-11           [-1, 16, 55, 55]           2,064
             ReLU-12           [-1, 16, 55, 55]               0
           Conv2d-13           [-1, 64, 55, 55]           1,088
             ReLU-14           [-1, 64,

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define transformations for the training and test sets
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomCrop(32, padding=4),  # Data augmentation
    transforms.ToTensor(),  # Convert PIL image to tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),  # Normalize
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Create data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

100.0%


In [5]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SqueezeNet(in_channels=3, num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(trainloader)}], Loss: {running_loss / 100:.4f}")
            running_loss = 0.0

print("Training finished.")

Epoch [1/10], Step [100/391], Loss: 2.3042
Epoch [1/10], Step [200/391], Loss: 2.2557
Epoch [1/10], Step [300/391], Loss: 2.0154
Epoch [2/10], Step [100/391], Loss: 1.9102
Epoch [2/10], Step [200/391], Loss: 1.8826
Epoch [2/10], Step [300/391], Loss: 1.8586
Epoch [3/10], Step [100/391], Loss: 1.8315
Epoch [3/10], Step [200/391], Loss: 1.8290
Epoch [3/10], Step [300/391], Loss: 1.7996
Epoch [4/10], Step [100/391], Loss: 1.7851
Epoch [4/10], Step [200/391], Loss: 1.7724
Epoch [4/10], Step [300/391], Loss: 1.7662
Epoch [5/10], Step [100/391], Loss: 1.7451
Epoch [5/10], Step [200/391], Loss: 1.7011
Epoch [5/10], Step [300/391], Loss: 1.6835
Epoch [6/10], Step [100/391], Loss: 1.6175
Epoch [6/10], Step [200/391], Loss: 1.6150
Epoch [6/10], Step [300/391], Loss: 1.5776
Epoch [7/10], Step [100/391], Loss: 1.5527
Epoch [7/10], Step [200/391], Loss: 1.5341
Epoch [7/10], Step [300/391], Loss: 1.5307
Epoch [8/10], Step [100/391], Loss: 1.5036
Epoch [8/10], Step [200/391], Loss: 1.4750
Epoch [8/10

In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Accuracy on the test set: 50.95%
